In [1]:
import random

[random.randint(8, 15) for i in range(2000)]

In [1]:
import pandas as pd

In [2]:
open_poses = pd.read_excel('data/tax/broker_statement.xlsx', sheet_name='open_poses_pd')
open_poses = open_poses[open_poses['ticker'] != 'RURCUR']

In [3]:
open_poses_2021 = pd.read_excel('data/tax/broker_statement.xlsx', sheet_name='open_poses_end2021')
open_poses_2021 = open_poses_2021[open_poses_2021['ticker'] != 'RURCUR']

In [4]:
trades_2022 = pd.read_excel('data/tax/broker_statement.xlsx', sheet_name='trades_pd')
trades_2022 = trades_2022[trades_2022['ticker'] != 'RURCUR']


In [5]:
open_poses.sample(5)

,ticker,count_start,price_start,volume_start,count_end,price_end,volume_end,balance
55,TMUS,42,115.98,4871.16,0,0.00,0.0,0.0
56,TOST_IPO,20,34.71,694.20,0,0.00,0.0,0.0
12,F.GOGR_USD,0,0.00,0.00,42,13.05,548.1,548.1
62,USER_IPO,66,8.42,555.72,0,0.00,0.0,0.0
65,VACC,33,11.11,366.63,0,0.00,0.0,0.0


In [6]:
trades_2022.sample(5)

,ticker,dir,price,count,total,com_brok,com_ex
327,TZA,Продажа,45.30,-20,906.00,1.20,0.00
534,SOXS,Продажа,35.01,-2,70.02,1.20,0.00
310,SOXS,Купля,40.33,100,-4033.00,1.20,0.03
165,JETS,Продажа,20.48,-454,9297.92,5.45,1.55
34,COHU,Продажа,33.35,-80,2668.00,1.20,-0.08


In [7]:
deriv_tickers = "UVXY TQQQ UPRO URTY KOLD SPXU TZA SOXS BOIL SQQQ TMF VXX NUGT JNUG YINN UNL".split(' ')

In [8]:
share_tickers = list(set(trades_2022['ticker']).union(set(open_poses['ticker'])).union(set(open_poses_2021['ticker'])) - set(deriv_tickers))

In [9]:
exclude_tickers = share_tickers

In [10]:
def filter_by_ticker(df, exclude_tickers):
    return df[~df['ticker'].isin(exclude_tickers)].copy()

In [11]:
def calc_deduct_expenses(poses, trades, exclude_tickers=[]):
    poses = filter_by_ticker(poses, exclude_tickers)
    trades = filter_by_ticker(trades, exclude_tickers)
    iterator = poses[poses['count_end'] != 0].iterrows()
    open_end = 0
    ixs = []
    tickers = []
    balance_sums = []
    trade_counts = []


    while True:
        try: 
            pose = next(iterator)
            purcahse_count = 0
            balance_sum = .0
            balance_price = pose[1]['balance'] / pose[1]['count_end']
            for row in trades[(trades['ticker'] == pose[1]['ticker']) &
                                     (trades['dir'] == 'Купля')].iterrows():

                purcahse_count += row[1]['count']
                balance_sum += row[1]['count'] * balance_price
                if purcahse_count > pose[1]['count_end']:
                    balance_sum -= (purcahse_count - pose[1]['count_end']) * balance_price
                    purcahse_count = pose[1]['count_end']
                if pose[1]['count_end'] == purcahse_count:
                    break
            tickers.append(pose[1]['ticker'])
            balance_sums.append(balance_sum)
            trade_counts.append(purcahse_count)

            open_end += 1
            ixs.append(pose[0])
        except StopIteration:
            break
    return pd.DataFrame(data={'ticker': tickers, 'balance': balance_sums, 'trade_count': trade_counts})

In [27]:
deduct_expenses = calc_deduct_expenses(open_poses, trades_2022[::-1], deriv_tickers)

In [13]:
deduct_expenses

,ticker,balance,trade_count
0,JNUG,1384.72,38
1,NUGT,3021.82,93
2,TMF,2769.77,357
3,TQQQ,1817.93,107
4,URTY,1380.08,37


In [14]:
def calc_deduct_profits(poses, trades, exclude_tickers=[]):
    poses = filter_by_ticker(poses, exclude_tickers)
    trades = filter_by_ticker(trades, exclude_tickers)
    iterator = poses[poses['count_end'] != 0].iterrows()
    open_end = 0
    ixs = []
    tickers = []
    balance_sums = []
    trade_counts = []

    while True:
        try: 
            pose = next(iterator)
            sell_count = 0
            balance_sum = .0
            balance_price = pose[1]['balance'] / pose[1]['count_end']
            for row in trades[(trades['ticker'] == pose[1]['ticker']) &
                                     (trades['dir'] == 'Продажа')].iterrows():

                sell_count += -row[1]['count']
                balance_sum += -row[1]['count'] * balance_price
                if sell_count > pose[1]['count_end']:
                    balance_sum -= (sell_count - pose[1]['count_end']) * balance_price
                    sell_count = pose[1]['count_end']
                if pose[1]['count_end'] == sell_count:
                    break
            tickers.append(pose[1]['ticker'])
            balance_sums.append(balance_sum)
            trade_counts.append(sell_count)

            open_end += 1
            ixs.append(pose[0])
        except StopIteration:
            break
    return pd.DataFrame(data={'ticker': tickers, 'balance': balance_sums, 'trade_count': trade_counts})

In [28]:
deduct_profits = calc_deduct_profits(open_poses_2021, trades_2022, deriv_tickers)

In [16]:
deduct_profits

,ticker,balance,trade_count
0,UVXY,14701.88,1036


In [23]:
def calc_profit_loss(trades, poses_current, poses_prev, exclude_tickers=[]):
    trades = filter_by_ticker(trades, exclude_tickers)
    reverse_trades = trades[::-1]
    deduct_profits = calc_deduct_profits(poses_prev, trades, exclude_tickers)
    deduct_expenses = calc_deduct_expenses(poses_current, reverse_trades, exclude_tickers)
    sales_only_mask = trades['dir'] == 'Продажа'
    purchases_only_mask = trades['dir'] == 'Купля'
    profit = trades[sales_only_mask]['total'].sum()
    expense = (-trades[purchases_only_mask]['total'].sum()) + \
                trades['com_brok'].sum() + \
                trades['com_ex'].sum() + \
                deduct_profits['balance'].sum() - \
                deduct_expenses['balance'].sum()
    return pd.DataFrame({'profit': [profit], 'expense': [expense], 'result': [profit - expense]})
            

In [47]:
calc_profit_loss(trades_2022, open_poses, open_poses_2021, share_tickers)

,profit,expense,result
0,733136.99,749856.21,-16719.22


In [55]:
interest_margin_total = 474.38

In [29]:
print(calc_trades_result(trades_2022, deriv_tickers) - deduct_profits['balance'].sum() + deduct_expenses['balance'].sum())

-11732.662063348445


In [48]:
749856.21 * 68.3522

51254321.63716199

In [45]:
15213810.260784589 - 14411856.996922 

801953.2638625894

In [ ]:
51254321.63716199

In [ ]:
50 111 526.16

In [ ]:
733 136.99

In [ ]:
68.3522

In [13]:
( 50_111_526.16 / 51254321.63716199) * 368.4231145245024

360.2085434305545

In [17]:
(14_411_856.99 / 15213810.25) * 105.95688547549757

100.37166597228195

In [20]:
68.3522 * 360.2085434305545 + 51254321.637161

51278942.68356328

In [ ]:
51254321.637161

In [54]:
15213810.25 + 50_111_526.16

65325336.41

In [52]:
14_411_856.99 + 801_953.26

15213810.25

In [ ]:
14 411 856.99

In [ ]:
801 953.26

In [ ]:
210 847.01

In [ ]:
68.3522

In [46]:

22000.00 * .13

2860.0

In [10]:
14_411_856.99/ (14_411_856.99 + 50_111_526.16) * 474.38

105.95688547549757

In [11]:
474.38 - 14_411_856.99/ (14_411_856.99 + 50_111_526.16) * 474.38

368.4231145245024

In [49]:
4000 + 1590 + 1300

6890

In [51]:
3752 / (4000 + 1590 + 1300 + 22000)

0.12987192800276912